# Get native contact list from crystal structure

In [ ]:
from Bio.PDB import PDBParser
from Bio.PDB import NeighborSearch
import numpy as np
from Bio.PDB import Selection

In [ ]:
# calculate the distance and out put the atom pair (heavy atom and i-j>4(at least 3 reaidue away), 4.5Å)
def get_distances(pdb_file):
    parser = PDBParser()
    pdb_structure = parser.get_structure("pdb_structure", pdb_file)
    model = pdb_structure[0]
    atom_list = [atom for atom in Selection.unfold_entities(model, 'A') if atom.get_id()[0] != 'H']
    distances = []
    pairs_atom = []
    for i in range(len(atom_list)):
        for j in range(i+1, len(atom_list)):
            if abs(atom_list[i].get_parent().get_id()[1] - atom_list[j].get_parent().get_id()[1]) < 4:
                continue
            distance = np.linalg.norm(atom_list[i].coord - atom_list[j].coord)
            if distance >= 4.5:
                continue
            distances.append((distance, atom_list[i].get_serial_number(),atom_list[i], atom_list[j].get_serial_number(),atom_list[j]))
            pairs_atom.append((atom_list[i].get_serial_number(),atom_list[j].get_serial_number(),distance))
    return pairs_atom

In [ ]:
Insulin_FNC = get_distances('./2g4m.pdb')

In [ ]:
# adjust the decimal places
Insulin_native_l = []
for i in range (0,len(Insulin_FNC)):
    Insulin_native_l.append(format(Insulin_FNC[i][2]/10,'.4f'))

weight_Insulin = format(1/len(Insulin_FNC),'.6f')

In [ ]:
# generate plumed.dat file
with open(".//plumed_FNC.dat","w") as f:
 print("""
CONTACTMAP ...""",file=f)
 for i in range(0, len(Insulin_FNC)):
    print(f"ATOMS{i+1}={Insulin_FNC[i][0]},{Insulin_FNC[i][1]} SWITCH{i+1}={{Q R_0=0.01 BETA=50.0 LAMBDA=1.8 REF={Insulin_native_l[i]}}} WEIGHT{i+1}={weight_Insulin}",file=f)
 print("""LABEL=cmap
SUM
... CONTACTMAP

PRINT ARG=* FILE=colvar STRIDE=1 """,file=f)

# Convert secondary structure data (from VMD timeline STRIDE) to dataframe for plot

In [ ]:
def load_and_process_SS_data(file_path):
    # Load txt file
    with open(file_path, 'r', encoding='utf8') as file:
        lines = file.readlines()
        
    # Skip the first 9 rows
    lines = lines[9:]
    
    # Convert into DataFrame
    df = pd.DataFrame([line.split() for line in lines])
    
    # Import data into DataFrame and drop unnecessary data
    df.columns = ["residue#", "chain_identifier", "empty", "frame", "secondary_structure"]
    SS = df.drop(['chain_identifier', 'empty'], axis=1)
    SS['residue#'] = pd.to_numeric(SS['residue#'])
    
    # Calculate start_residue and end_residue
    start_residue = SS['residue#'].min()
    end_residue = SS['residue#'].max()
    
    # Sort for secondary structure data
    Lr = []
    for i in range(start_residue, end_residue + 1):
        Lr_temp = SS.loc[SS["residue#"] == i]['secondary_structure'].reset_index(drop=True)
        Lr.append(Lr_temp)

    # Convert into DataFrame
    ss = pd.DataFrame(Lr)
    ss = ss.set_index(pd.Index(range(start_residue, end_residue + 1)))
    
    return ss

In [ ]:
ss_INS  = load_and_process_SS_data('./INS_300K_rt1.txt')

In [ ]:
ss_dict={}
def SimplifySecStructure(ssl):
  """
  This function takes in a list ssl of standard dssp secondary structure letters (G,H,I,E,B,T,C,S)
  and returns a simplified list where each element is assigned to the broader category of helix, beta or coil (H,E,C).
  So it basically maps H to H; E to E; H,I,B,T,C and S to C.
  """
  ss_dict={}
  for el in ['H']:ss_dict[el]='H'
  for el in ['E']:ss_dict[el]='E'
  for el in ['T','C','S','I','G','B']:ss_dict[el]='C'  
        
  return [ss_dict[ss] for ss in ssl]

In [ ]:
def SimplifySecStructureList(ss_list):
  """
  This function takes a list of lists of secondary structures as obtained from AnalyzeSecondaryStructure
  and reassigns each element to the broader categories of Helix, Extended and Coil (see SimplifySecStructure)
  Inputs:
  ss_list: A list of lists of secondary structures as obtained from AnalyzeSecondaryStructure, see SecStrucColorMap.
  Outpu:
  ss_list: Simplified list of lists of secondary structures
  """
  return [SimplifySecStructure(ssl) for ssl in ss_list]

In [ ]:
# WT 300K
ss_dict1 = []

for i in range(0, 50001, 10):
    ss_dict_temp2 = []
    ss_dict_temp = ss_INS[i]
    ss_dict_temp = SimplifySecStructureList(ss_dict_temp)
    for sublist in ss_dict_temp:
        for item in sublist:
            ss_dict_temp2.append(item)
    ss_dict1.append(ss_dict_temp2)

for i in range(0, 5001):
    for j in range(0, 52):  # Make sure that this range is appropriate
        ss_dict1[i][j] = ss_label_mapping.get(ss_dict1[i][j], ss_dict1[i][j])

ss_full = pd.DataFrame(ss_dict1)
ss_full.index = range(0, 50001,10)
ss_map2 = ss_full.transpose().to_numpy()

In [ ]:
fig,ax1=plt.subplots(1,figsize=(10,5),dpi=300)

ax1.imshow(ss_map2,cmap = "gnuplot2",interpolation='nearest',aspect='auto')

xtick_labels= [0,20,40,60,80,100] # manualy define the x tick labels
xtick_positons = [0,1000,2000,3000,4000,5000] # position of the points
ax1.set_xticks(xtick_positons)
ax1.set_yticks(ytick_positons)
ax1.set_xticklabels(xtick_labels,fontsize=28)
ax1.set_yticklabels(ytick_labels,fontsize=28)
ax1.tick_params(direction="in")
ax1.set_ylabel('Residue Index', fontsize=34)
ax1.set_xlabel('Time(ns)', fontsize=34)
for axis in ['top', 'bottom', 'left', 'right']:
    ax1.spines[axis].set_linewidth(1.5)